ver2: 로그 씌운 후 비율% 계산
- data_1901_2004_classified_2_outliered -> 전처리 -> processed_data_1901_2004_ver2

# ---자동화---
- 3시간32분

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import numpy as np
import unicodedata
import os
import time
from tqdm import tqdm

data_dir = '/content/drive/My Drive/영빈/data/'
data = pd.read_feather(data_dir + 'data_1901_2004_classified_2_outliered.ftr', #
                       columns=['년', '월', '가맹점소재지1', '가맹점소재지2', '성별', '연령대별', '연평균소득추정','이용금액', '이용건수', '업종']) # index col 빼고 읽어오기


""" --------------- 날짜 루프 ------------------ """


date_list = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912,
             202001, 202002, 202003, 202004]

all_data = []
from tqdm import tqdm
for date in tqdm(date_list):

  year = int(str(date)[:4]) # 2019
  month = str(date)[4:] # 01

  if month[0] == '0':
    month = int(month[1]) # 1
  else:
    month = int(month) 
  
  """ 데이터 월별 필터링 """

  df = data[(data['년']==year) & (data['월']==month)]
  print('월별 데이터: ',df.shape)


  """ --------------- 전처리 ------------------ """


  """ 고객 그룹별 업종 이용금액 """

  grouped = df.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정', '업종']) 
  grouped_sum = grouped['이용금액'].sum()
  df_spend_part = pd.DataFrame(grouped_sum)

  # Calculate logarithm on column 
  df_spend_part['이용금액'] = np.log10(df_spend_part['이용금액'])
  df_spend_part

  """ 고객 그룹별 전체 이용금액 (로그 씌운 업종 이용금액들의 합) """

  grouped = df_spend_part.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정']) 
  grouped_sum = grouped['이용금액'].sum()
  df_spend_total = pd.DataFrame(grouped_sum)
  df_spend_total


  """ 이용금액 % 생성 """

  from tqdm import tqdm
  for index,row in df_spend_part.iterrows():
      
      sort_list = list(index)[:-1] # e.g., 2019년 1월 서울시 노원구 남성 20대 B1 (업종제외)
      year, month, region1, region2, gender, age, income = sort_list[0], sort_list[1], sort_list[2], sort_list[3], sort_list[4], sort_list[5], sort_list[6]
      spend_total = df_spend_total.loc[     (df_spend_total.index.get_level_values('년') == year) 
                                          & (df_spend_total.index.get_level_values('월') == month) 
                                          & (df_spend_total.index.get_level_values('가맹점소재지1') == region1)   
                                          & (df_spend_total.index.get_level_values('가맹점소재지2') == region2)
                                          & (df_spend_total.index.get_level_values('성별') == gender) 
                                          & (df_spend_total.index.get_level_values('연령대별') == age)
                                          & (df_spend_total.index.get_level_values('연평균소득추정') == income)]['이용금액']
      
      spend_total = float(spend_total)    
      df_spend_part.at[index, '총 이용금액'] = spend_total
      
  df_spend_part['이용금액%'] = df_spend_part['이용금액'] / df_spend_part['총 이용금액'] 

  """ 데이터셋 형태로 만들기 """

  df_final = df_spend_part.copy()
  df_final.drop(['이용금액', '총 이용금액'], axis=1, inplace=True)
  df_final = df_final.reset_index(level=[0])
  df_final = pd.pivot_table(df_final, values='이용금액%', columns='업종', 
                                    index=['년','월','가맹점소재지1','가맹점소재지2', '성별', '연령대별', '연평균소득추정'])
  df_final = df_final.fillna(0) # NaN -> 0
  df_final["합계%"] = df_final.sum(axis=1)

  """ reverse dict """ 

  # load data
  import pickle
  with open(data_dir + 'dict_pickle/region1_dict.pickle', 'rb') as fr:
      region1_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/region2_dict.pickle', 'rb') as fr:
      region2_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/gender_dict.pickle', 'rb') as fr:
      gender_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/age_dict.pickle', 'rb') as fr:
      age_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/income_dict.pickle', 'rb') as fr:
      income_dict = pickle.load(fr)
  with open(data_dir + 'dict_pickle/store_dict.pickle', 'rb') as fr:
      store_dict = pickle.load(fr)

  region1_dict = {value : key for (key, value) in region1_dict.items()}
  region2_dict = {value : key for (key, value) in region2_dict.items()}
  gender_dict = {value : key for (key, value) in gender_dict.items()}
  age_dict = {value : key for (key, value) in age_dict.items()}
  income_dict = {value : key for (key, value) in income_dict.items()}
  store_dict = {value : key for (key, value) in store_dict.items()}


  df_final.rename(index = region1_dict, level=2, inplace=True)
  df_final.rename(index = region2_dict, level=3, inplace=True)
  df_final.rename(index = gender_dict, level=4, inplace=True)
  df_final.rename(index = age_dict, level=5, inplace=True)
  df_final.rename(index = income_dict, level=6, inplace=True)
  df_final.rename(columns = store_dict, inplace = True)
  df_final.reset_index(inplace=True)

  print('최종 데이터: ', df_final.shape)

  all_data.append(df_final)

frame = pd.concat(all_data, axis=0, ignore_index=True)
frame.to_feather(data_dir + 'processed_data_1901_2004_ver2.ftr')

Mounted at /content/drive


  0%|          | 0/16 [00:00<?, ?it/s]

월별 데이터:  (5286559, 10)


  6%|▋         | 1/16 [11:39<2:54:54, 699.63s/it]

최종 데이터:  (20033, 26)
월별 데이터:  (5126968, 10)


...

최종 데이터:  (18499, 26)
월별 데이터:  (4980641, 10)


 94%|█████████▍| 15/16 [3:06:36<11:50, 710.01s/it]

최종 데이터:  (19757, 26)
월별 데이터:  (5255586, 10)


100%|██████████| 16/16 [3:18:25<00:00, 744.09s/it]

최종 데이터:  (20101, 26)


In [1]:
""" 데이터 확인 """

from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/영빈/data/'
pd.read_feather(data_dir + 'clustering_data_1901_2004_ver2.ftr')

Mounted at /content/drive


,년,월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,가정용품,개인미용,개인운송,교육서비스,기타,대형판매,보건,보험,숙박서비스,식료품,여객운송,오락_문화,음식점,의류_잡화,전자상거래,정보통신,종합소매,주거,합계%
0,2019,1,강원도,강릉시,남성,20대,B1,0.000000,0.080753,0.083550,0.0,0.070243,0.089342,0.074931,0.000000,0.085634,0.083187,0.000000,0.086330,0.101798,0.076967,0.0,0.000000,0.096038,0.071226,1.0
1,2019,1,강원도,강릉시,남성,20대,B2,0.056414,0.067953,0.072221,0.0,0.063494,0.063584,0.071995,0.000000,0.070145,0.070962,0.046565,0.072593,0.081565,0.070055,0.0,0.050118,0.079024,0.063310,1.0
2,2019,1,강원도,강릉시,남성,20대,B3,0.062628,0.066407,0.072482,0.0,0.060411,0.071310,0.065679,0.000000,0.067773,0.069860,0.059990,0.070744,0.080057,0.066608,0.0,0.047542,0.076211,0.062296,1.0
3,2019,1,강원도,강릉시,남성,20대,B4,0.050855,0.069511,0.078441,0.0,0.066329,0.077052,0.071600,0.000000,0.073826,0.074778,0.061831,0.074278,0.086668,0.069605,0.0,0.000000,0.081791,0.063435,1.0
4,2019,1,강원도,강릉시,남성,20대,B5,0.000000,0.072664,0.080242,0.0,0.070913,0.092891,0.069117,0.000000,0.087678,0.085726,0.000000,0.086786,0.105210,0.081301,0.0,0.000000,0.097668,0.069804,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337029,2020,4,충청북도,충주시,여성,60대 이상,B3,0.071159,0.070057,0.076747,0.0,0.064504,0.075626,0.078873,0.000000,0.062651,0.074568,0.000000,0.075388,0.077108,0.071733,0.0,0.053238,0.080063,0.068284,1.0
337030,2020,4,충청북도,충주시,여성,60대 이상,B4,0.065214,0.066518,0.072612,0.0,0.065949,0.070601,0.074468,0.054986,0.058681,0.070663,0.000000,0.074097,0.073751,0.066663,0.0,0.046633,0.075815,0.063349,1.0
337031,2020,4,충청북도,충주시,여성,60대 이상,B5,0.000000,0.000000,0.111932,0.0,0.000000,0.117114,0.110120,0.000000,0.000000,0.102331,0.000000,0.122893,0.120572,0.109966,0.0,0.000000,0.123454,0.081617,1.0
337032,2020,4,충청북도,충주시,여성,60대 이상,B6,0.000000,0.000000,0.000000,0.0,0.000000,0.204812,0.185327,0.000000,0.000000,0.000000,0.000000,0.217693,0.206517,0.000000,0.0,0.000000,0.185651,0.000000,1.0


# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import numpy as np
import unicodedata
import os
import time
from tqdm import tqdm

data_dir = '/content/drive/My Drive/Colab Notebooks/개인연구/data/'
data = pd.read_feather(data_dir + 'data_1901_2004_classified_2.ftr', # 88656835 rows
                       columns=['년', '월', '가맹점소재지1', '가맹점소재지2', '성별', '연령대별', '연평균소득추정','이용금액', '이용건수', '업종']) # index col 빼고 읽어오기
data.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88656835 entries, 0 to 88656834
Data columns (total 10 columns):
 #   Column   Dtype  
---  ------   -----  
 0   년        int16  
 1   월        int8   
 2   가맹점소재지1  int8   
 3   가맹점소재지2  int16  
 4   성별       int8   
 5   연령대별     int8   
 6   연평균소득추정  int8   
 7   이용금액     float32
 8   이용건수     int32  
 9   업종       int8   
dtypes: float32(1), int16(2), int32(1), int8(6)
memory usage: 1.5 GB


In [ ]:
data

,년,월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,이용금액,이용건수,업종
0,2019,1,0,3,1,3,3,1070000.0,4,18
1,2019,1,0,3,0,0,3,48000.0,4,5
2,2019,1,0,3,0,4,5,735000.0,4,1
3,2019,1,0,3,0,2,5,1041289.0,16,14
4,2019,1,0,3,1,3,5,426766.0,13,14
...,...,...,...,...,...,...,...,...,...,...
88656830,2020,4,16,183,1,4,6,303000.0,17,9
88656831,2020,4,16,183,1,5,0,100000.0,5,9
88656832,2020,4,16,183,1,5,3,89000.0,7,9
88656833,2020,4,16,183,1,5,4,636000.0,29,9


# 데이터 월별 필터링

In [ ]:
""" 2020년 4월 데이터로 연습 """

df = data[(data['년'] == 2020) & (data['월'] == 4)]
print(df.shape)
sorted(df.업종.unique())

(5336937, 10)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

# 전처리
- col: 업종 20개
  - 식당, 자동차서비스, 보건위생, 식당, 주점 등
- row: 고객 그룹
  - 지역, 성별, 연령대별, 연평균소득추정 
- value: 이용금액(%)

In [ ]:
""" 고객 그룹별 업종 이용금액 """

grouped = df.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정', '업종']) 
grouped_sum = grouped['이용금액'].sum()
df_spend_part = pd.DataFrame(grouped_sum)

# Calculate logarithm on column 
df_spend_part['이용금액'] = np.log10(df_spend_part['이용금액'])
df_spend_part

이용금액
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정 업종          
2020 4 0       3       0  0    0       1   5.106562
                                       2   6.132490
                                       3   6.458940
                                       5   5.214844
                                       6   5.091315
...                                             ...
       16      183     1  5    7       3   5.933285
                                       9   5.628675
                                       12  5.051153
                                       17  5.059980
                               8       9   4.687083

[217756 rows x 1 columns]

In [ ]:
""" 고객 그룹별 전체 이용금액 (로그 씌운 업종 이용금액들의 합) """

grouped = df_spend_part.groupby(['년', '월', '가맹점소재지1', '가맹점소재지2', '성별','연령대별', '연평균소득추정']) 
grouped_sum = grouped['이용금액'].sum()
df_spend_total = pd.DataFrame(grouped_sum)
df_spend_total

이용금액
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정            
2020 4 0       3       0  0    0         82.491470
                               3        127.923744
                               4        136.681732
                               5        117.721809
                               6         75.404739
...                                            ...
       16      183     1  5    4        124.694191
                               5        121.256287
                               6         54.052574
                               7         27.255302
                               8          4.687083

[20101 rows x 1 columns]

In [ ]:
""" 이용금액 % 생성 """

from tqdm import tqdm
for index,row in tqdm(df_spend_part.iterrows()):
    
    sort_list = list(index)[:-1] # e.g., 2019년 1월 서울시 노원구 남성 20대 B1 (업종제외)
    year, month, region1, region2, gender, age, income = sort_list[0], sort_list[1], sort_list[2], sort_list[3], sort_list[4], sort_list[5], sort_list[6]
    spend_total = df_spend_total.loc[     (df_spend_total.index.get_level_values('년') == year) 
                                        & (df_spend_total.index.get_level_values('월') == month) 
                                        & (df_spend_total.index.get_level_values('가맹점소재지1') == region1)   
                                        & (df_spend_total.index.get_level_values('가맹점소재지2') == region2)
                                        & (df_spend_total.index.get_level_values('성별') == gender) 
                                        & (df_spend_total.index.get_level_values('연령대별') == age)
                                        & (df_spend_total.index.get_level_values('연평균소득추정') == income)]['이용금액']
    
    spend_total = float(spend_total)    
    df_spend_part.at[index, '총 이용금액'] = spend_total
    
df_spend_part['이용금액%'] = df_spend_part['이용금액'] / df_spend_part['총 이용금액'] 
df_spend_part

217756it [14:35, 248.77it/s]


이용금액     총 이용금액     이용금액%
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정 업종                               
2020 4 0       3       0  0    0       1   5.106562  82.491470  0.061904
                                       2   6.132490  82.491470  0.074341
                                       3   6.458940  82.491470  0.078298
                                       5   5.214844  82.491470  0.063217
                                       6   5.091315  82.491470  0.061719
...                                             ...        ...       ...
       16      183     1  5    7       3   5.933285  27.255302  0.217693
                                       9   5.628675  27.255302  0.206517
                                       12  5.051153  27.255302  0.185327
                                       17  5.059980  27.255302  0.185651
                               8       9   4.687083   4.687083  1.000000

[217756 rows x 3 columns]

In [ ]:
""" 데이터셋 형태로 만들기 """

df_final = df_spend_part.copy()
df_final.drop(['이용금액', '총 이용금액'], axis=1, inplace=True)
df_final = df_final.reset_index(level=[0])
df_final = pd.pivot_table(df_final, values='이용금액%', columns='업종', 
                                   index=['년','월','가맹점소재지1','가맹점소재지2', '성별', '연령대별', '연평균소득추정'])
df_final = df_final.fillna(0) # NaN -> 0
df_final["합계%"] = df_final.sum(axis=1)
df_final

업종                                             0         1  ...        19  합계%
년    월 가맹점소재지1 가맹점소재지2 성별 연령대별 연평균소득추정                      ...               
2020 4 0       3       0  0    0        0.000000  0.061904  ...  0.000000  1.0
                               3        0.044619  0.050871  ...  0.042867  1.0
                               4        0.044184  0.048338  ...  0.040727  1.0
                               5        0.000000  0.053390  ...  0.037842  1.0
                               6        0.000000  0.000000  ...  0.000000  1.0
...                                          ...       ...  ...       ...  ...
       16      183     1  5    4        0.058772  0.057113  ...  0.000000  1.0
                               5        0.057335  0.059499  ...  0.000000  1.0
                               6        0.000000  0.000000  ...  0.000000  1.0
                               7        0.000000  0.000000  ...  0.000000  1.0
                               8        0.000000  0.000000  ...  0.000000  1.0

[20101 rows x 21 columns]

In [ ]:
""" reverse dict """ 

# load data
import pickle
with open(data_dir + 'dict_pickle/region1_dict.pickle', 'rb') as fr:
    region1_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/region2_dict.pickle', 'rb') as fr:
    region2_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/gender_dict.pickle', 'rb') as fr:
    gender_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/age_dict.pickle', 'rb') as fr:
    age_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/income_dict.pickle', 'rb') as fr:
    income_dict = pickle.load(fr)
with open(data_dir + 'dict_pickle/store_dict.pickle', 'rb') as fr:
    store_dict = pickle.load(fr)

region1_dict = {value : key for (key, value) in region1_dict.items()}
region2_dict = {value : key for (key, value) in region2_dict.items()}
gender_dict = {value : key for (key, value) in gender_dict.items()}
age_dict = {value : key for (key, value) in age_dict.items()}
income_dict = {value : key for (key, value) in income_dict.items()}
store_dict = {value : key for (key, value) in store_dict.items()}


df_final.rename(index = region1_dict, level=2, inplace=True)
df_final.rename(index = region2_dict, level=3, inplace=True)
df_final.rename(index = gender_dict, level=4, inplace=True)
df_final.rename(index = age_dict, level=5, inplace=True)
df_final.rename(index = income_dict, level=6, inplace=True)
df_final.rename(columns = store_dict, inplace = True)
df_final.reset_index(inplace=True)
df_final

업종,년,월,가맹점소재지1,가맹점소재지2,성별,연령대별,연평균소득추정,"가구,가전",기타,대형판매,레져,"문화,취미","보건,위생",서비스,"서적,문구",쇼핑,식당,식료품,"여행,숙박","의료,보험",인테리어,자동차,자동차서비스,전자상거래,종합소매,주점,학원,합계%
0,2020,4,강원도,강릉시,남성,20대,B1,0.000000,0.061904,0.074341,0.078298,0.000000,0.063217,0.061719,0.064276,0.071227,0.085344,0.070414,0.066200,0.070275,0.000000,0.000000,0.072339,0.000000,0.083156,0.077291,0.000000,1.0
1,2020,4,강원도,강릉시,남성,20대,B2,0.044619,0.050871,0.057068,0.059128,0.047770,0.054526,0.050819,0.052141,0.055473,0.064259,0.056000,0.053987,0.055357,0.040130,0.000000,0.057257,0.034764,0.063232,0.059732,0.042867,1.0
2,2020,4,강원도,강릉시,남성,20대,B3,0.044184,0.048338,0.055406,0.055999,0.048146,0.052191,0.048473,0.047799,0.052505,0.061876,0.054670,0.053703,0.054226,0.037221,0.037134,0.056259,0.034739,0.060024,0.056380,0.040727,1.0
3,2020,4,강원도,강릉시,남성,20대,B4,0.000000,0.053390,0.061030,0.061588,0.046986,0.056109,0.050573,0.053562,0.057063,0.069113,0.060559,0.059924,0.058408,0.033978,0.048848,0.062156,0.000000,0.066439,0.062432,0.037842,1.0
4,2020,4,강원도,강릉시,남성,20대,B5,0.000000,0.000000,0.083324,0.075049,0.000000,0.069362,0.062185,0.065318,0.072546,0.093059,0.078913,0.076808,0.076463,0.000000,0.000000,0.077653,0.000000,0.089626,0.079694,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20096,2020,4,충청북도,충주시,여성,60대 이상,B3,0.058772,0.057113,0.062779,0.062144,0.054624,0.059070,0.052887,0.048256,0.060961,0.064098,0.061901,0.052008,0.065475,0.054287,0.000000,0.063710,0.000000,0.066462,0.055450,0.000000,1.0
20097,2020,4,충청북도,충주시,여성,60대 이상,B4,0.057335,0.059499,0.062493,0.065415,0.053981,0.058879,0.052469,0.048300,0.060987,0.065106,0.062548,0.051942,0.065945,0.048294,0.000000,0.065290,0.000000,0.067109,0.054409,0.000000,1.0
20098,2020,4,충청북도,충주시,여성,60대 이상,B5,0.000000,0.000000,0.116522,0.122272,0.000000,0.000000,0.081205,0.000000,0.109410,0.119963,0.101814,0.000000,0.114572,0.000000,0.000000,0.111411,0.000000,0.122830,0.000000,0.000000,1.0
20099,2020,4,충청북도,충주시,여성,60대 이상,B6,0.000000,0.000000,0.204812,0.217693,0.000000,0.000000,0.000000,0.000000,0.000000,0.206517,0.000000,0.000000,0.185327,0.000000,0.000000,0.000000,0.000000,0.185651,0.000000,0.000000,1.0
